In [13]:
import openai
import os

from pprint import pprint

import pandas as pd
from dotenv import load_dotenv, find_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

_ = load_dotenv(find_dotenv())
openai.api_key = os.getenv('OPENAI_API_KEY')

In [5]:
df = pd.read_csv('../data/product_reviews.csv')
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


# LLMChain

In [14]:
# High temp -> more creative names
llm = ChatOpenAI(temperature=0.9)
prompt = ChatPromptTemplate.from_template(
    'What is the best name to describe a company that sells {product}?',
)
chain = LLMChain(llm=llm, prompt=prompt)

for product in df.Product.iloc[:5]:
    company_name_idea = chain.run(product)
    print(f"Product: {product}, Generated name: {company_name_idea}")

Product: Queen Size Sheet Set, Generated name: "Royal Dreams Linens" or "Regal Comfort Bedding"
Product: Waterproof Phone Pouch, Generated name: Aquashield
Product: Luxury Air Mattress, Generated name: 1. Opulent Rest
2. LuxeSlumber
3. Supreme Sleep
4. Elite Air Mattress Co.
5. DreamSerenity
6. LuxAir Mattress
7. PlushDreams
8. PrestigeSleep
9. RegalAir Mattress
10. SereneSlumber
Product: Pillows Insert, Generated name: The Cushion Corner
Product: Milk Frother Handheld
, Generated name: "Milk Frother Plus" or "FrothMaster"


In [20]:
# You can also generate multiple results at once like this
products = list(df.Product.iloc[:5])
products = [{'product': product} for product in products]
chain.generate(products)

LLMResult(generations=[[ChatGeneration(text='RoyalComfort Beddings', generation_info=None, message=AIMessage(content='RoyalComfort Beddings', additional_kwargs={}, example=False))], [ChatGeneration(text='AquaGuard Phone Accessories', generation_info=None, message=AIMessage(content='AquaGuard Phone Accessories', additional_kwargs={}, example=False))], [ChatGeneration(text='OpulentRest', generation_info=None, message=AIMessage(content='OpulentRest', additional_kwargs={}, example=False))], [ChatGeneration(text='PillowPerfect', generation_info=None, message=AIMessage(content='PillowPerfect', additional_kwargs={}, example=False))], [ChatGeneration(text='FrothMaster', generation_info=None, message=AIMessage(content='FrothMaster', additional_kwargs={}, example=False))]], llm_output={'token_usage': {'prompt_tokens': 116, 'completion_tokens': 19, 'total_tokens': 135}, 'model_name': 'gpt-3.5-turbo'})

Note: you can use OpenAI via your Azure deployment like so

```python
from langchain.llms import AzureOpenAI

llm = AzureOpenAI(
deployment_name="your-azure-deployment",
model_name="text-davinci-003"
)
```

# SimpleSequentialChain

The simplest form of sequential chains, where each step has a singular input/output, and the output of one step is the input to the next.

In [12]:
llm = ChatOpenAI(temperature=0.9)

first_prompt = ChatPromptTemplate.from_template(
    'What is the best name to describe a company that sells {product}?',
)
first_chain = LLMChain(llm=llm, prompt=first_prompt)

second_prompt = ChatPromptTemplate.from_template(
    'Write a 20 word description for the following company: {company_name}',
)
second_chain = LLMChain(llm=llm, prompt=second_prompt)

overall_simple_chain = SimpleSequentialChain(
    chains=[first_chain, second_chain],
    verbose=True,
)

for product in df.Product.iloc[:5]:
    overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Royal Comfort Beddings
Royal Comfort Beddings is a luxury bedding brand offering top-quality, stylish bedding sets and accessories for a blissful sleep experience.

> Finished chain.


> Entering new SimpleSequentialChain chain...
AquaGuard Phone Protection
AquaGuard Phone Protection offers high-quality waterproof cases and screen protectors to ensure maximum protection for your valuable smartphones.

> Finished chain.


> Entering new SimpleSequentialChain chain...
Opulent Rest
Opulent Rest offers luxurious and high-quality bedding products that provide the ultimate comfort and relaxation for a restful night's sleep.

> Finished chain.


> Entering new SimpleSequentialChain chain...
PillowPro
PillowPro offers premium and customizable pillows designed for optimal comfort and support, ensuring a restful night's sleep.

> Finished chain.


> Entering new SimpleSequentialChain chain...
FrothMaster
FrothMaster is a leading manufacturer of hig

# SequentialChain

A more general form of sequential chains, allowing for multiple inputs/outputs.

In [ ]:
llm = ChatOpenAI(temperature=0.9)

first_prompt = ChatPromptTemplate.from_template(
    'Translate the following review to English: {review}',
)
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key='english_review')

second_prompt = ChatPromptTemplate.from_template(
    'Summarize the following review in 1 sentence: {english_review}',
)
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key='summary')

third_prompt = ChatPromptTemplate.from_template(
    'What language is the following review: {review}'
)
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key='language')

fourth_prompt = ChatPromptTemplate.from_template(
    'What a follow up response to the following summary in the specified'
    'language: Summary: {summary}\n\n Language: {language}'
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key='follow_up_response')

overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=['review'],
    output_variables=['english_review', 'summary', 'follow_up_response'],
    verbose=True,
)